#### **Fundamentos de Bancos de Dados Relacionais e NoSQL**

#### **Conteúdo - Bases e Notebook da aula**

https://github.com/FIAP/Pos_Tech_DTAT/tree/main/Fase%203

#### **Importação de pacotes, bibliotecas e funções (def)**

In [22]:
#!pip uninstall boto3 -y
!pip uninstall botocore -y
# !pip install boto3==1.40.26 
# !pip install botocore==1.40.26
# # !pip install s3fs



Found existing installation: botocore 1.40.26
Uninstalling botocore-1.40.26:
  Successfully uninstalled botocore-1.40.26


In [23]:
#!pip install boto3==1.40.26 
!pip install botocore==1.40.26

  Using cached botocore-1.40.26-py3-none-any.whl.metadata (5.7 kB)
Using cached botocore-1.40.26-py3-none-any.whl (14.0 MB)


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 2.12.3 requires botocore<1.34.70,>=1.34.41, but you have botocore 1.40.26 which is incompatible.


In [24]:
import boto3, botocore
print("boto3:", boto3.__version__)
print("botocore:", botocore.__version__)


boto3: 1.40.26
botocore: 1.40.26


In [ ]:
# Importar biblioteca completa
import boto3
import pandas as pd
import os
import plotly.express as px
import requests
import botocore
import psycopg2
import numpy as np
import duckdb
import mongomock
import fakeredis
import uuid
import json

# Importar função especifica de um módulo
from botocore.exceptions import BotoCoreError, ClientError
from sqlalchemy import create_engine, text, inspect
from dotenv import load_dotenv
from io import StringIO,BytesIO
from datetime import datetime
from pprint import pprint
from astrapy import DataAPIClient
from pathlib import Path
from urllib.parse import urlparse

#### **Testar conexão AWS via Python**

In [26]:
# Validar conexão
try:
    session = boto3.Session(profile_name="default")
    sts = session.client("sts")
    identity = sts.get_caller_identity()
    print("✅ Conectado à conta\n")
    print("UserId:", identity["UserId"])
    print("Account:", identity["Account"])
    print("Arn:", identity["Arn"])

except (BotoCoreError, ClientError) as e:
    print("❌ Erro ao conectar à AWS. Verifique suas credenciais e tente novamente.")
    print("Detalhes do erro:", e)


✅ Conectado à conta

UserId: AROAZI2LFHB2TERRDYNA4:user4377774=maradinidiego16@gmail.com
Account: 637423401077
Arn: arn:aws:sts::637423401077:assumed-role/voclabs/user4377774=maradinidiego16@gmail.com


##### **Conectar ao PostgreSQL via RDS + Executar Comandos SQL**

In [27]:

load_dotenv("Users\dmaradini\.aws\credentials")

aws_access_key_id       = os.getenv("aws_access_key_id")
aws_secret_access_key   = os.getenv("aws_secret_access_key")
aws_session_token       = os.getenv("aws_session_token")  

<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
C:\Users\dmaradini\AppData\Local\Temp\ipykernel_18620\2708774549.py:1: SyntaxWarning: invalid escape sequence '\d'
  load_dotenv("Users\dmaradini\.aws\credentials")


In [28]:
import boto3
from botocore.exceptions import ClientError
import json

session = boto3.Session(
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
    aws_session_token=aws_session_token
)

bucket_name = "techchallange-637423401077"
s3 = session.client("s3")

# Descobrir região da sessão; se não houver, use us-east-1
region = session.region_name or s3.meta.region_name or "us-east-1"

# 1) Verificar se o bucket existe e é seu/acessível
def bucket_exists_and_accessible(name: str) -> bool:
    try:
        s3.head_bucket(Bucket=name)
        return True
    except ClientError as e:
        code = e.response.get("Error", {}).get("Code", "")
        # 404/NoSuchBucket => não existe; 301 => região diferente; 403 => existe mas sem acesso
        if code in ("404", "NoSuchBucket"):
            return False
        elif code in ("301", "PermanentRedirect"):
            # Existe em outra região; ainda dá para usar, mas crie/use cliente na região correta
            return True
        elif code in ("403", "AccessDenied"):
            # Bucket provavelmente existe mas não é seu (nome global do S3)
            raise RuntimeError(
                f"Bucket '{name}' já existe, mas você não tem acesso. "
                "Escolha outro nome (o namespace do S3 é global)."
            )
        else:
            raise

# 2) Criar bucket SEM ACLs (compatível com ObjectOwnership: BucketOwnerEnforced)
if bucket_exists_and_accessible(bucket_name):
    print(f"Bucket '{bucket_name}' já existe e está acessível.")
else:
    print(f"Bucket '{bucket_name}' não existe; criando...")
    if region == "us-east-1":
        # Em us-east-1 não envia CreateBucketConfiguration
        s3.create_bucket(Bucket=bucket_name)
    else:
        s3.create_bucket(
            Bucket=bucket_name,
            CreateBucketConfiguration={"LocationConstraint": region},
        )
    print(f"Bucket '{bucket_name}' criado com sucesso.")

# 3) Ajustar Public Access Block:
#    Para permitir bucket policy pública, desative os bloqueios de policy no BUCKET.
#    (Os bloqueios de ACL podem continuar como True, já que não usamos ACLs.)
s3.put_public_access_block(
    Bucket=bucket_name,
    PublicAccessBlockConfiguration={
        "BlockPublicAcls": True,      # pode ficar True (ACLs desativadas por padrão)
        "IgnorePublicAcls": True,     # idem
        "BlockPublicPolicy": False,   # precisa ser False para aceitar policy pública
        "RestrictPublicBuckets": False
    }
)

# 4) Aplicar bucket policy pública de leitura de objetos
public_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "AllowPublicReadOfObjects",
            "Effect": "Allow",
            "Principal": "*",
            "Action": ["s3:GetObject"],
            "Resource": f"arn:aws:s3:::{bucket_name}/*"
        }
    ]
}

s3.put_bucket_policy(Bucket=bucket_name, Policy=json.dumps(public_policy))
print(f"Bucket '{bucket_name}' configurado para leitura pública via Bucket Policy.")


Bucket 'techchallange-637423401077' já existe e está acessível.
Bucket 'techchallange-637423401077' configurado para leitura pública via Bucket Policy.


In [29]:
import pandas as pd
import boto3
import numpy as npfrp
from datetime import datetime
from io import StringIO
from botocore.exceptions import ClientError


session = boto3.Session(
    aws_access_key_id       =   aws_access_key_id
    ,aws_secret_access_key  =   aws_secret_access_key
    ,aws_session_token      =   aws_session_token
)



bucket_name = "techchallange-637423401077"
s3_prefix = "raw"



print("Conexão estabelecida.")

s3 = session.client('s3')
region = s3.meta.region_name or "us-east-1"

try:
    s3.head_bucket(Bucket=bucket_name)
    print(f"Bucket '{bucket_name}' já existe e está acessível.")
except ClientError as e:
    error_code = e.response['Error']['Code']
    if error_code in ("404", "NoSuchBucket"):
        print(f"Bucket '{bucket_name}' não existe, criando...\n")
        if region == "us-east-1":
            s3.create_bucket(Bucket=bucket_name)
            
        else:
            s3.create_bucket(
                Bucket=bucket_name,
                CreateBucketConfiguration={'LocationConstraint': region}
            )
        print(f"Bucket '{bucket_name}' criado com sucesso.\n")
    else:
        print(f"Erro ao acessar o bucket: {e}")
        raise



# Criar política de bucket pública
public_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "PublicReadGetObject",
            "Effect": "Allow",
            "Principal": "*",
            "Action": "s3:GetObject",
            "Resource": f"arn:aws:s3:::{bucket_name}/*"
        }
    ]
}

s3.put_bucket_policy(Bucket=bucket_name, Policy=json.dumps(public_policy))
print(f"Bucket '{bucket_name}' configurado como público.")

Conexão estabelecida.
Bucket 'techchallange-637423401077' já existe e está acessível.
Bucket 'techchallange-637423401077' configurado como público.


### CAMADA BRONZE (EXTRACT)

In [30]:
import pandas as pd
import boto3
from io import BytesIO, StringIO
from zipfile import ZipFile
import requests

bucket_name = "techchallange-637423401077"
s3_prefix = "raw"

s3 = boto3.client("s3")

# URLs dos arquivos do IBGE
urls = {
    "PNAD_COVID_052020": "https://ftp.ibge.gov.br/Trabalho_e_Rendimento/Pesquisa_Nacional_por_Amostra_de_Domicilios_PNAD_COVID19/Microdados/Dados/PNAD_COVID_052020.zip",
    "PNAD_COVID_062020": "https://ftp.ibge.gov.br/Trabalho_e_Rendimento/Pesquisa_Nacional_por_Amostra_de_Domicilios_PNAD_COVID19/Microdados/Dados/PNAD_COVID_062020.zip",
    "PNAD_COVID_072020": "https://ftp.ibge.gov.br/Trabalho_e_Rendimento/Pesquisa_Nacional_por_Amostra_de_Domicilios_PNAD_COVID19/Microdados/Dados/PNAD_COVID_072020.zip",
}

for tabela, url in urls.items():
    print(f"Processando {tabela}...")

    r = requests.get(url)
    r.raise_for_status()

    if "Dicionario" in tabela:
        # Salvar o XLS original no S3
        s3_key = f"{s3_prefix}/{tabela}.xls"
        s3.put_object(
            Bucket=bucket_name,
            Key=s3_key,
            Body=r.content  # conteúdo binário do XLS
        )
        print(f"Dicionário salvo no S3 em s3://{bucket_name}/{s3_key}")

    else:
        # Abrir o ZIP e extrair o CSV
        with ZipFile(BytesIO(r.content)) as z:
            csv_name = z.namelist()[0]
            with z.open(csv_name) as f:
                df = pd.read_csv(f, sep=";", encoding="latin1")

        # Converter para CSV em memória
        csv_buffer = StringIO()
        df.to_csv(csv_buffer, index=False)

        s3_key = f"{s3_prefix}/{tabela}.csv"
        s3.put_object(
            Bucket=bucket_name,
            Key=s3_key,
            Body=csv_buffer.getvalue()
        )
        print(f"Dados salvos no S3 em s3://{bucket_name}/{s3_key}")

print("Exportação concluída com sucesso.")


Processando PNAD_COVID_052020...
Dados salvos no S3 em s3://techchallange-637423401077/raw/PNAD_COVID_052020.csv
Processando PNAD_COVID_062020...
Dados salvos no S3 em s3://techchallange-637423401077/raw/PNAD_COVID_062020.csv
Processando PNAD_COVID_072020...
Dados salvos no S3 em s3://techchallange-637423401077/raw/PNAD_COVID_072020.csv
Exportação concluída com sucesso.


In [31]:
import boto3

session = boto3.Session(profile_name="default")
s3 = session.client("s3")
print("Consegui conectar no S3!")


Consegui conectar no S3!


### CAMADA SILVER (TRANFORM)

In [32]:
import pandas as pd
import requests
from io import StringIO

# Lista de arquivos CSV no bucket público
bronze_files = [
    "PNAD_COVID_052020.csv",
    "PNAD_COVID_062020.csv",
    "PNAD_COVID_072020.csv"
]

# URL base do bucket público
base_url = "https://techchallange-637423401077.s3.us-east-1.amazonaws.com/raw/"

lista_de_dataframes = []

for file_name in bronze_files:
    file_url = f"{base_url}{file_name}"
    print(f"Lendo {file_url}")
    
    # Faz download do CSV como texto
    response = requests.get(file_url)
    response.raise_for_status()  # garante que deu certo
    
    # Lê o CSV como texto cru
    df_raw = pd.read_csv(StringIO(response.text), header=None)
    
    # Divide a única coluna pelo separador vírgula
    df_temp = df_raw[0].str.split(",", expand=True)
    
    # Define a primeira linha como header
    df_temp.columns = df_temp.iloc[0]
    df_temp = df_temp.drop(0).reset_index(drop=True)
    
    lista_de_dataframes.append(df_temp)

# Consolidar todos os dataframes
df_consolidado = pd.concat(lista_de_dataframes, ignore_index=True)


Lendo https://techchallange-637423401077.s3.us-east-1.amazonaws.com/raw/PNAD_COVID_052020.csv
Lendo https://techchallange-637423401077.s3.us-east-1.amazonaws.com/raw/PNAD_COVID_062020.csv
Lendo https://techchallange-637423401077.s3.us-east-1.amazonaws.com/raw/PNAD_COVID_072020.csv


In [33]:
import boto3
import pandas as pd
from io import BytesIO

# Configurações do bucket
bucket_name = "techchallange-637423401077"
s3_key = "silver/df_consolidado.parquet"

# Conexão S3
s3_client = boto3.client("s3")  # precisa das credenciais configuradas no seu ambiente

# Salva o DataFrame em um buffer Parquet
buffer = BytesIO()
df_consolidado.to_parquet(buffer, index=False)

# Envia para o S3
buffer.seek(0)
s3_client.put_object(Bucket=bucket_name, Key=s3_key, Body=buffer.getvalue())

print(f"DataFrame salvo em s3://{bucket_name}/{s3_key}")


DataFrame salvo em s3://techchallange-637423401077/silver/df_consolidado.parquet


In [34]:
df_consolidado_parquet_aws = pd.read_parquet("https://techchallange-637423401077.s3.us-east-1.amazonaws.com/silver/df_consolidado.parquet")
#pd.read_parquet('Dados.parquet') #

In [35]:
df_consolidado_parquet = df_consolidado_parquet_aws

In [36]:
import pandas as pd

# Dicionário de renomeação completo
mapa_colunas = {
    # Colunas fixas
    "ANO": "ano",
    "V1013": "mes_pesquisa",
    "V1012": "semana_mes",
    "UF": "uf",
    "CAPITAL": "capital",

    # Colunas desejadas
    "A002": "idade",
    "A003": "sexo",
    "A004": "raca_cor",
    "A006B": "aulas_presenciais",
    "B008": "fez_exame",
    "B009A": "exame_swab",
    "B009C": "exame_furo_dedo",
    "B009E": "exame_veia_braco",
    "A005": "escolaridade",
    "A006": "frequenta_escola",
    "A006A": "tipo_escola",
    "B0011": "teve_febre_semana_passada",
    "B0012": "teve_tosse_semana_passada",
    "B0013": "teve_dor_garganta_semana_passada",
    "B0014": "teve_dificuldade_respirar",
    "B0015": "teve_dor_cabeca",
    "B0016": "teve_dor_peito",
    "B0017": "teve_nausea",
    "B0018": "teve_nariz_entupido",
    "B0019": "teve_fadiga",
    "B00110": "teve_dor_olhos",
    "B00111": "teve_perda_cheiro_sabor",
    "B00112": "teve_dor_muscular",
    "B00113": "teve_diarreia"
}


colunas_existentes = [col for col in mapa_colunas.keys() if col in df_consolidado_parquet.columns]

# Renomeia apenas essas colunas
df_consolidado_parquet.rename(columns={col: mapa_colunas[col] for col in colunas_existentes}, inplace=True)


df_consolidado_parquet = df_consolidado_parquet[[mapa_colunas[col] for col in colunas_existentes]]



In [37]:
mapa_sim_nao = {
    1: "Sim",
    2: "Não",
    3: "Não sabe",
    9: "Ignorado"
}

mapa_uf = {
    11: "Rondônia", 12: "Acre", 13: "Amazonas", 14: "Roraima", 15: "Pará", 16: "Amapá", 17: "Tocantins",
    21: "Maranhão", 22: "Piauí", 23: "Ceará", 24: "Rio Grande do Norte", 25: "Paraíba", 26: "Pernambuco",
    27: "Alagoas", 28: "Sergipe", 29: "Bahia", 31: "Minas Gerais", 32: "Espírito Santo", 33: "Rio de Janeiro",
    35: "São Paulo", 41: "Paraná", 42: "Santa Catarina", 43: "Rio Grande do Sul", 50: "Mato Grosso do Sul",
    51: "Mato Grosso", 52: "Goiás", 53: "Distrito Federal"
}


df_consolidado_parquet = df_consolidado_parquet.copy()

df_consolidado_parquet["uf"] = df_consolidado_parquet["uf"].astype(int)
df_consolidado_parquet["uf"] = df_consolidado_parquet["uf"].replace(mapa_uf)




# # Substituir colunas de Sim/Não (exemplo: febre, tosse, exames, etc)
colunas_sim_nao = [
    "fez_exame", "exame_swab", "exame_furo_dedo", "exame_veia_braco",
    "teve_febre_semana_passada", "teve_tosse_semana_passada", "teve_dor_garganta_semana_passada",
    "teve_dificuldade_respirar", "teve_dor_cabeca", "teve_dor_peito", "teve_nausea",
    "teve_nariz_entupido", "teve_fadiga", "teve_dor_olhos", "teve_perda_cheiro_sabor",
    "teve_dor_muscular", "teve_diarreia", "aulas_presenciais", "frequenta_escola"
]
colunas_sim_nao = [
    "fez_exame", "exame_swab", "exame_furo_dedo", "exame_veia_braco",
    "teve_febre_semana_passada", "teve_tosse_semana_passada", "teve_dor_garganta_semana_passada",
    "teve_dificuldade_respirar", "teve_dor_cabeca", "teve_dor_peito", "teve_nausea",
    "teve_nariz_entupido", "teve_fadiga", "teve_dor_olhos", "teve_perda_cheiro_sabor",
    "teve_dor_muscular", "teve_diarreia", "aulas_presenciais", "frequenta_escola"
]

# Criar um dicionário de substituição com strings
mapa_sim_nao_str = {str(k): v for k, v in mapa_sim_nao.items()}

for col in colunas_sim_nao:
    if col in df_consolidado_parquet.columns:
        # Substituir valores apenas se não forem nulos
        df_consolidado_parquet[col] = df_consolidado_parquet[col].apply(
            lambda x: mapa_sim_nao_str.get(str(x), x) if pd.notna(x) else x
        )

# Dicionário de substituição
mapa_sexo = {
    "1": "Homem",
    "2": "Mulher"
}

# Substituir valores na coluna 'sexo' (somente se a coluna existir)
if "sexo" in df_consolidado_parquet.columns:
    df_consolidado_parquet["sexo"] = df_consolidado_parquet["sexo"].astype(str).replace(mapa_sexo)


# Dicionário de mapeamento para raca_cor
mapa_raca_cor = {
    "1": "Branca",
    "2": "Preta",
    "3": "Amarela",
    "4": "Parda",
    "5": "Indígena",
    "9": "Ignorado"
}

# Aplicar mapeamento no dataframe
if "raca_cor" in df_consolidado_parquet.columns:
    df_consolidado_parquet["raca_cor"] = df_consolidado_parquet["raca_cor"].astype(str).replace(mapa_raca_cor)



In [38]:
df_consolidado_parquet["sexo"] = df_consolidado_parquet["sexo"].astype("category")
df_consolidado_parquet["mes_pesquisa"] = df_consolidado_parquet["mes_pesquisa"].astype("category")
df_consolidado_parquet["raca_cor"] = df_consolidado_parquet["raca_cor"].astype("category")


In [39]:
import boto3
import pandas as pd
from io import BytesIO

# Seu DataFrame
# df_consolidado

# Configurações do bucket
bucket_name = "techchallange-637423401077"
s3_key = "silver/df_consolidado_Tratado.parquet"

# Conexão S3
s3_client = boto3.client("s3")  # precisa das credenciais configuradas no seu ambiente

# Salva o DataFrame em um buffer Parquet
buffer = BytesIO()
df_consolidado_parquet.to_parquet(buffer, index=False)

# Envia para o S3
buffer.seek(0)
s3_client.put_object(Bucket=bucket_name, Key=s3_key, Body=buffer.getvalue())

print(f"DataFrame salvo em s3://{bucket_name}/{s3_key}")


DataFrame salvo em s3://techchallange-637423401077/silver/df_consolidado_Tratado.parquet


### CAMADA OURO (LOAD)


In [ ]:
INPUT_PATH = "https://techchallange-637423401077.s3.us-east-1.amazonaws.com/silver/df_consolidado_Tratado.parquet"

# S3 de DESTINO (GOLD)
bucket_name = "techchallange-637423401077"
prefix = "gold/"  # "pasta" no bucket (terminar com /)

SINTOMA_PREFIX = "teve_"  # colunas de sintomas começam com isso

# =========================
# S3 helpers
# =========================
s3_client = boto3.client("s3")

def save_parquet_s3(df: pd.DataFrame, key: str):
    """
    Salva um DataFrame em formato Parquet no S3 (sem passar por disco),
    no bucket/prefix configurados acima.
    """
    buf = BytesIO()
    df.to_parquet(buf, index=False, engine="pyarrow")
    buf.seek(0)
    s3_client.put_object(Bucket=bucket_name, Key=key, Body=buf.getvalue())
    print(f"[OK] s3://{bucket_name}/{key} ({len(df):,} linhas)")

# =========================
# IO (entrada) - URL HTTP e fallback boto3
# =========================
def _read_parquet_http(url: str) -> pd.DataFrame:
    """Lê um .parquet via HTTP(S). Se falhar e for URL S3, tenta fallback via boto3.get_object()."""
    try:
        r = requests.get(url, timeout=120)
        r.raise_for_status()
        return pd.read_parquet(BytesIO(r.content), engine="pyarrow")
    except Exception as e_http:
        # Fallback: tenta boto3 se a URL for do S3 (objeto privado, por ex.)
        try:
            parsed = urlparse(url)
            # Ex.: techchallange-...s3.us-east-1.amazonaws.com/silver/df.parquet
            key = parsed.path.lstrip("/")                # silver/df_consolidado_Tratado.parquet
            bucket = parsed.netloc.split(".")[0]         # techchallange-637423401077
            obj = s3_client.get_object(Bucket=bucket, Key=key)
            body = obj["Body"].read()
            return pd.read_parquet(BytesIO(body), engine="pyarrow")
        except Exception as e_s3:
            raise RuntimeError(f"Falha ao ler {url} (HTTP e fallback S3). HTTP: {e_http} | S3: {e_s3}")

def read_any(path: str) -> pd.DataFrame:
    """Lê CSV/Parquet local, diretório com vários Parquets, OU URL HTTP(S)."""
    if path.lower().startswith(("http://", "https://")):
        if path.lower().endswith(".parquet"):
            return _read_parquet_http(path)
        elif path.lower().endswith(".csv"):
            r = requests.get(path, timeout=120)
            r.raise_for_status()
            return pd.read_csv(BytesIO(r.content), sep=";", low_memory=False)
        else:
            raise ValueError("URL deve apontar para .parquet ou .csv")
    # Local (opcional)
    p = Path(path)
    if p.is_dir():
        files = [str(x) for x in p.rglob("*.parquet")]
        if not files:
            raise FileNotFoundError(f"Nenhum .parquet em {p}")
        dfs = [pd.read_parquet(f, engine="pyarrow") for f in files]
        return pd.concat(dfs, ignore_index=True)
    suf = p.suffix.lower()
    if suf == ".csv":
        return pd.read_csv(p, sep=";", low_memory=False)
    if suf == ".parquet":
        return pd.read_parquet(p, engine="pyarrow")
    raise ValueError("Forneça .csv, .parquet, diretório contendo .parquet, ou URL http(s) para .parquet/.csv")

df = read_any(INPUT_PATH)

# =========================
# NORMALIZAÇÕES
# =========================
def to_bool_series(s: pd.Series) -> pd.Series:
    if s is None:
        return None
    if s.dtype == bool:
        return s.astype("boolean")
    t = s.astype(str).str.strip().str.lower()
    true_vals = {"1","true","t","sim","s","y","yes"}
    false_vals = {"0","false","f","nao","não","n","no"}
    out = pd.Series(pd.NA, index=s.index, dtype="boolean")
    out = out.mask(t.isin(true_vals), True).mask(t.isin(false_vals), False)
    return out

# Idade
if "idade" in df.columns:
    df["idade"] = pd.to_numeric(df["idade"], errors="coerce")
    df = df[(df["idade"].notna()) & (df["idade"] >= 0) & (df["idade"] <= 110)]

# Sexo
if "sexo" in df.columns:
    df["sexo"] = df["sexo"].astype(str).str.strip().str.title()
    mapa_sexo = {"Homem":"Homem","Masculino":"Homem","M":"Homem",
                 "Mulher":"Mulher","Feminino":"Mulher","F":"Mulher"}
    df["sexo"] = df["sexo"].map(lambda x: mapa_sexo.get(x, x))

# UF / Raça
if "uf" in df.columns:
    df["uf"] = df["uf"].astype(str).str.strip().str.upper()
if "raca_cor" in df.columns:
    df["raca_cor"] = df["raca_cor"].astype(str).str.strip().str.title()

# Econômicas
for col in ["trabalhou_semana_passada", "teletrabalho"]:
    if col in df.columns:
        df[col] = to_bool_series(df[col])
if "rendimento" in df.columns:
    df["rendimento"] = pd.to_numeric(df["rendimento"], errors="coerce")
if "situacao_ocupacao" in df.columns:
    df["situacao_ocupacao"] = df["situacao_ocupacao"].astype(str).str.strip().str.title()

# Faixas etárias
if "idade" in df.columns:
    bins = [0, 12, 18, 25, 35, 45, 60, 75, np.inf]
    labels = ["0–12","13–18","19–25","26–35","36–45","46–60","61–75","75+"]
    df["faixa_etaria"] = pd.cut(df["idade"], bins=bins, labels=labels, right=True, include_lowest=True)

# Sintomas
sintomas_cols = [c for c in df.columns if c.startswith(SINTOMA_PREFIX)]
for c in sintomas_cols:
    df[c] = to_bool_series(df[c])

# =========================
# BASE CURADA
# =========================
save_parquet_s3(df, prefix + "base_curada.parquet")

# =========================
# POPULAÇÃO
# =========================
if "sexo" in df.columns:
    tab = df["sexo"].value_counts(dropna=False).reset_index()
    tab.columns = ["sexo","qtd"]
    save_parquet_s3(tab, prefix + "pop_sexo.parquet")

if "faixa_etaria" in df.columns:
    tab = df["faixa_etaria"].value_counts().sort_index().reset_index()
    tab.columns = ["faixa_etaria","qtd"]
    save_parquet_s3(tab, prefix + "pop_faixa_etaria.parquet")

if "raca_cor" in df.columns:
    tab = df["raca_cor"].value_counts(dropna=False).reset_index()
    tab.columns = ["raca_cor","qtd"]
    save_parquet_s3(tab, prefix + "pop_raca_cor.parquet")

if "uf" in df.columns:
    top_uf = df["uf"].value_counts().head(10).reset_index()
    top_uf.columns = ["uf","qtd"]
    save_parquet_s3(top_uf, prefix + "pop_top10_uf.parquet")

if {"uf","sexo"}.issubset(df.columns):
    tab = df.groupby(["uf","sexo"]).size().reset_index(name="qtd")
    top_ufs = df["uf"].value_counts().head(10).index
    tab = tab[tab["uf"].isin(top_ufs)]
    save_parquet_s3(tab, prefix + "pop_sexo_por_uf.parquet")

# =========================
# CLÍNICO – SINTOMAS
# =========================
if sintomas_cols:
    # Prevalência (%) geral por sintoma
    prev = (df[sintomas_cols].mean(numeric_only=True) * 100.0).reset_index()
    prev.columns = ["sintoma_raw","perc"]
    prev["sintoma"] = (prev["sintoma_raw"].str.replace("^teve_","",regex=True)
                       .str.replace("_semana_passada$","",regex=True)
                       .str.replace("_"," ").str.title())
    save_parquet_s3(prev[["sintoma","perc"]], prefix + "cli_prevalencia_sintomas.parquet")

    # Nº de sintomas por pessoa (severidade)
    sint_bool = df[sintomas_cols].astype("boolean")
    n_sint = sint_bool.sum(axis=1).value_counts().sort_index().reset_index()
    n_sint.columns = ["n_sintomas","qtd"]
    save_parquet_s3(n_sint, prefix + "cli_n_sintomas_hist.parquet")

    # Coocorrência (Spearman) – wide
    corr = sint_bool.fillna(False).astype(int).corr(method="spearman")
    corr_wide = corr.reset_index().rename(columns={"index":"sintoma"})
    save_parquet_s3(corr_wide, prefix + "cli_coocorrencia_spearman_wide.parquet")

    # Tosse / Febre por sexo (contagem e %)
    foco = {
        "tosse": [c for c in sintomas_cols if "tosse" in c],
        "febre": [c for c in sintomas_cols if "febre" in c],
    }
    if "sexo" in df.columns:
        base_sexo = df.groupby("sexo").size().rename("total").reset_index()
        for nome, cols in foco.items():
            if not cols:
                continue
            any_col = df[cols].any(axis=1)
            cont = any_col.groupby(df["sexo"]).sum().reset_index(name="qtd")
            save_parquet_s3(cont, prefix + f"cli_{nome}_contagem_por_sexo.parquet")
            perc = cont.merge(base_sexo, on="sexo")
            perc["perc"] = (perc["qtd"] / perc["total"]) * 100.0
            save_parquet_s3(perc, prefix + f"cli_{nome}_percentual_por_sexo.parquet")

# =========================
# ECONÔMICO
# =========================
if "situacao_ocupacao" in df.columns:
    tab = df["situacao_ocupacao"].value_counts(dropna=False).reset_index()
    tab.columns = ["situacao_ocupacao","qtd"]
    save_parquet_s3(tab, prefix + "eco_situacao_ocupacao.parquet")

if "trabalhou_semana_passada" in df.columns:
    tab = df["trabalhou_semana_passada"].value_counts(dropna=False).reset_index()
    tab.columns = ["valor","qtd"]
    save_parquet_s3(tab, prefix + "eco_trabalhou_semana.parquet")

if "teletrabalho" in df.columns:
    tab = df["teletrabalho"].value_counts(dropna=False).reset_index()
    tab.columns = ["valor","qtd"]
    save_parquet_s3(tab, prefix + "eco_teletrabalho.parquet")

if "rendimento" in df.columns:
    save_parquet_s3(df[["rendimento"]].dropna(), prefix + "eco_rendimento_base.parquet")

# =========================
# TEMPORAL
# =========================
time_col = next((c for c in ["mes_pesquisa","semana_mes"] if c in df.columns), None)

if time_col and sintomas_cols:
    def any_cols(cols): return df[cols].any(axis=1) if cols else pd.Series([False]*len(df), index=df.index)
    febre_cols = [c for c in sintomas_cols if "febre" in c]
    tosse_cols = [c for c in sintomas_cols if "tosse" in c]
    cabeca_cols = [c for c in sintomas_cols if "dor_cabeca" in c or "cabeca" in c]

    tmp = df[[time_col]].copy()
    tmp["sx_febre"] = any_cols(febre_cols)
    tmp["sx_tosse"] = any_cols(tosse_cols)
    tmp["sx_cabeca"] = any_cols(cabeca_cols)

    trend = (tmp.groupby(time_col)[["sx_febre","sx_tosse","sx_cabeca"]]
               .mean()*100.0).reset_index()
    trend_long = trend.melt(id_vars=time_col, var_name="sintoma", value_name="perc")
    trend_long["sintoma"] = trend_long["sintoma"].str.replace("sx_","",regex=False).str.title()
    save_parquet_s3(trend_long, prefix + "tmp_tendencia_sintomas.parquet")

print(f"\n[ETL] Concluído. Entrada via URL e agregados em s3://{bucket_name}/{prefix}")


[OK] s3://techchallange-637423401077/gold/base_curada.parquet (1,114,733 linhas)
[OK] s3://techchallange-637423401077/gold/pop_sexo.parquet (2 linhas)
[OK] s3://techchallange-637423401077/gold/pop_faixa_etaria.parquet (8 linhas)
[OK] s3://techchallange-637423401077/gold/pop_raca_cor.parquet (6 linhas)
[OK] s3://techchallange-637423401077/gold/pop_top10_uf.parquet (10 linhas)
[OK] s3://techchallange-637423401077/gold/pop_sexo_por_uf.parquet (20 linhas)
[OK] s3://techchallange-637423401077/gold/cli_prevalencia_sintomas.parquet (13 linhas)
[OK] s3://techchallange-637423401077/gold/cli_n_sintomas_hist.parquet (14 linhas)
[OK] s3://techchallange-637423401077/gold/cli_coocorrencia_spearman_wide.parquet (13 linhas)
[OK] s3://techchallange-637423401077/gold/cli_tosse_contagem_por_sexo.parquet (2 linhas)
[OK] s3://techchallange-637423401077/gold/cli_tosse_percentual_por_sexo.parquet (2 linhas)
[OK] s3://techchallange-637423401077/gold/cli_febre_contagem_por_sexo.parquet (2 linhas)
[OK] s3://tec

C:\Users\dmaradini\AppData\Local\Temp\ipykernel_18620\2031992308.py:239: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  trend = (tmp.groupby(time_col)[["sx_febre","sx_tosse","sx_cabeca"]]


[OK] s3://techchallange-637423401077/gold/tmp_tendencia_sintomas.parquet (9 linhas)

[ETL] Concluído. Entrada via URL e agregados em s3://techchallange-637423401077/gold/


### ANALISE DOS DADOS

In [45]:


import os
from io import BytesIO
import requests
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

BASE_URL = "https://techchallange-637423401077.s3.us-east-1.amazonaws.com/gold/"

OUT_DIR = "..\Graficos"
os.makedirs(OUT_DIR, exist_ok=True)

sns.set_theme(context="notebook", style="whitegrid")
sns.set_palette("mako") 


PARQUETS = {
    # POPULAÇÃO
    "pop_sexo": "pop_sexo.parquet",
    "pop_faixa_etaria": "pop_faixa_etaria.parquet",
    "pop_raca_cor": "pop_raca_cor.parquet",
    "pop_top10_uf": "pop_top10_uf.parquet",
    "pop_sexo_por_uf": "pop_sexo_por_uf.parquet",
    # CLÍNICO
    "cli_prevalencia_sintomas": "cli_prevalencia_sintomas.parquet",
    "cli_n_sintomas_hist": "cli_n_sintomas_hist.parquet",
    "cli_tosse_contagem_por_sexo": "cli_tosse_contagem_por_sexo.parquet",
    "cli_tosse_percentual_por_sexo": "cli_tosse_percentual_por_sexo.parquet",
    "cli_febre_contagem_por_sexo": "cli_febre_contagem_por_sexo.parquet",
    "cli_febre_percentual_por_sexo": "cli_febre_percentual_por_sexo.parquet",
    "cli_coocorrencia_spearman_wide": "cli_coocorrencia_spearman_wide.parquet",
    # ECONÔMICO
    "eco_situacao_ocupacao": "eco_situacao_ocupacao.parquet",
    "eco_trabalhou_semana": "eco_trabalhou_semana.parquet",
    "eco_teletrabalho": "eco_teletrabalho.parquet",
    "eco_rendimento_base": "eco_rendimento_base.parquet",
    # TEMPORAL
    "tmp_tendencia_sintomas": "tmp_tendencia_sintomas.parquet",
    "base_curada": "base_curada.parquet",
}

# ===============================
# HELPERS
# ===============================
def build_url(base: str, fname: str) -> str:
    if not base.endswith("/"):
        base = base + "/"
    return base + fname

def read_parquet_http(url: str) -> pd.DataFrame | None:
    """
    Baixa um .parquet via HTTP(S) e devolve um DataFrame.
    Retorna None se o arquivo não existir (HTTP 404) ou der erro de rede.
    """
    try:
        r = requests.get(url, timeout=60)
        if r.status_code != 200:
            print(f"[WARN] {url} -> HTTP {r.status_code}")
            return None
        bio = BytesIO(r.content)
        return pd.read_parquet(bio, engine="pyarrow")
    except Exception as e:
        print(f"[WARN] falha ao ler {url}: {e}")
        return None

def savefig(fig, name):
    path = os.path.join(OUT_DIR, name)
    fig.tight_layout()
    fig.savefig(path, bbox_inches="tight", dpi=120)
    plt.close(fig)
    print("fig:", path)

# ===============================
# LEITURA (via URL)
# ===============================
dfs = {}
for k, fname in PARQUETS.items():
    url = build_url(BASE_URL, fname)
    df = read_parquet_http(url)
    if df is None:
        print(f"[INFO] Pulando {k} (não encontrado em {url})")
    else:
        dfs[k] = df

# ===============================
# GRÁFICOS – POPULAÇÃO
# ===============================
if (tab := dfs.get("pop_sexo")) is not None and not tab.empty:
    fig, ax = plt.subplots(figsize=(7, 4))
    sns.barplot(data=tab.sort_values("qtd"), y="sexo", x="qtd", ax=ax)  # horizontal
    ax.set_title("Distribuição por Sexo")
    ax.set_xlabel("Quantidade"); ax.set_ylabel("Sexo")
    savefig(fig, "pop_1_sexo.png")

if (tab := dfs.get("pop_faixa_etaria")) is not None and not tab.empty:
    fig, ax = plt.subplots(figsize=(8, 5))
    tab["faixa_etaria"] = tab["faixa_etaria"].astype(str)
    sns.barplot(data=tab.sort_values("qtd"), y="faixa_etaria", x="qtd", ax=ax)
    ax.set_title("Distribuição por Faixa Etária")
    ax.set_xlabel("Quantidade"); ax.set_ylabel("Faixa Etária")
    savefig(fig, "pop_2_faixa_etaria.png")

if (tab := dfs.get("pop_raca_cor")) is not None and not tab.empty:
    fig, ax = plt.subplots(figsize=(9, 5))
    sns.barplot(data=tab.sort_values("qtd"), y="raca_cor", x="qtd", ax=ax)
    ax.set_title("Distribuição por Raça/Cor")
    ax.set_xlabel("Quantidade"); ax.set_ylabel("Raça/Cor")
    savefig(fig, "pop_3_raca_cor.png")

if (tab := dfs.get("pop_top10_uf")) is not None and not tab.empty:
    fig, ax = plt.subplots(figsize=(7, 6))
    sns.barplot(data=tab.sort_values("qtd"), y="uf", x="qtd", ax=ax)
    ax.set_title("Top 10 UFs (Contagem)")
    ax.set_xlabel("Quantidade"); ax.set_ylabel("UF")
    savefig(fig, "pop_4_top10_uf.png")

if (tab := dfs.get("pop_sexo_por_uf")) is not None and not tab.empty:
    fig, ax = plt.subplots(figsize=(10, 6))
    sns.barplot(data=tab.sort_values("qtd"), y="uf", x="qtd", hue="sexo", ax=ax)
    ax.set_title("Sexo por UF (Top 10)")
    ax.set_xlabel("Quantidade"); ax.set_ylabel("UF")
    ax.legend(title="Sexo")
    savefig(fig, "pop_5_sexo_por_uf.png")

# ===============================
# GRÁFICOS – CLÍNICO
# ===============================
if (tab := dfs.get("cli_prevalencia_sintomas")) is not None and not tab.empty:
    fig, ax = plt.subplots(figsize=(12, 6))
    sns.barplot(data=tab.sort_values("perc"), y="sintoma", x="perc", ax=ax)
    ax.set_title("Prevalência de Sintomas (%) – Geral")
    ax.set_xlabel("Percentual (%)"); ax.set_ylabel("Sintoma")
    savefig(fig, "cli_1_prevalencia_sintomas.png")

if (tab := dfs.get("cli_n_sintomas_hist")) is not None and not tab.empty:
    fig, ax = plt.subplots(figsize=(8, 5))
    sns.barplot(data=tab, y="n_sintomas", x="qtd", ax=ax)  # horizontal
    ax.set_title("Distribuição do Número de Sintomas")
    ax.set_xlabel("Quantidade"); ax.set_ylabel("Nº de Sintomas")
    savefig(fig, "cli_2_n_sintomas_hist.png")

for nome in ["tosse", "febre"]:
    if (cont := dfs.get(f"cli_{nome}_contagem_por_sexo")) is not None and not cont.empty:
        fig, ax = plt.subplots(figsize=(6, 4))
        sns.barplot(data=cont.sort_values("qtd"), y="sexo", x="qtd", ax=ax)
        ax.set_title(f"{nome.title()} – Contagem por Sexo")
        ax.set_xlabel("Contagem"); ax.set_ylabel("Sexo")
        savefig(fig, f"cli_{nome}_contagem_por_sexo.png")

    if (perc := dfs.get(f"cli_{nome}_percentual_por_sexo")) is not None and not perc.empty:
        fig, ax = plt.subplots(figsize=(6, 4))
        sns.barplot(data=perc.sort_values("perc"), y="sexo", x="perc", ax=ax)
        ax.set_title(f"{nome.title()} – Percentual por Sexo")
        ax.set_xlabel("Percentual (%)"); ax.set_ylabel("Sexo")
        savefig(fig, f"cli_{nome}_percentual_por_sexo.png")

# Heatmap de coocorrência
if (corr_wide := dfs.get("cli_coocorrencia_spearman_wide")) is not None and not corr_wide.empty:
    corr = corr_wide.set_index("sintoma")
    fig, ax = plt.subplots(figsize=(10, 8))
    sns.heatmap(corr, cmap="mako", ax=ax)
    ax.set_title("Coocorrência de Sintomas (Spearman)")
    savefig(fig, "cli_coocorrencia_heatmap.png")

# ===============================
# GRÁFICOS – ECONÔMICO
# ===============================
if (tab := dfs.get("eco_situacao_ocupacao")) is not None and not tab.empty:
    fig, ax = plt.subplots(figsize=(10, 5))
    sns.barplot(data=tab.sort_values("qtd"), y="situacao_ocupacao", x="qtd", ax=ax)
    ax.set_title("Situação de Ocupação")
    ax.set_xlabel("Quantidade"); ax.set_ylabel("Situação")
    savefig(fig, "eco_1_situacao_ocupacao.png")

if (tab := dfs.get("eco_trabalhou_semana")) is not None and not tab.empty:
    map_lab = {True: "Sim", False: "Não"}
    if "valor" in tab.columns:
        tab["label"] = tab["valor"].map(map_lab).fillna("NA")
    elif "label" not in tab.columns:
        tab["label"] = tab.iloc[:, 0].map(map_lab).fillna("NA")
    fig, ax = plt.subplots(figsize=(6, 4))
    sns.barplot(data=tab.sort_values("qtd"), y="label", x="qtd", ax=ax)
    ax.set_title("Trabalhou na Semana Passada")
    ax.set_xlabel("Quantidade"); ax.set_ylabel("")
    savefig(fig, "eco_2_trabalhou_semana.png")

if (tab := dfs.get("eco_teletrabalho")) is not None and not tab.empty:
    map_lab = {True: "Sim", False: "Não"}
    if "valor" in tab.columns:
        tab["label"] = tab["valor"].map(map_lab).fillna("NA")
    elif "label" not in tab.columns:
        tab["label"] = tab.iloc[:, 0].map(map_lab).fillna("NA")
    fig, ax = plt.subplots(figsize=(6, 4))
    sns.barplot(data=tab.sort_values("qtd"), y="label", x="qtd", ax=ax)
    ax.set_title("Teletrabalho")
    ax.set_xlabel("Quantidade"); ax.set_ylabel("")
    savefig(fig, "eco_3_teletrabalho.png")

if (rend := dfs.get("eco_rendimento_base")) is not None and not rend.empty:
    fig, ax = plt.subplots(figsize=(9, 5))
    sns.histplot(data=rend, x="rendimento", bins=30, ax=ax)
    ax.set_title("Distribuição de Rendimento (R$)")
    ax.set_xlabel("R$"); ax.set_ylabel("Frequência")
    savefig(fig, "eco_4_rendimento_hist.png")

# ===============================
# GRÁFICOS – TEMPORAL
# ===============================
if (trend := dfs.get("tmp_tendencia_sintomas")) is not None and not trend.empty:
    # trend deve ter colunas: [<periodo>, "sintoma", "perc"]
    tcol = trend.columns[0] if trend.columns[0] not in ["sintoma", "perc"] else "periodo"
    if tcol not in trend.columns:
        trend = trend.rename(columns={trend.columns[0]: "periodo"})
        tcol = "periodo"
    fig, ax = plt.subplots(figsize=(11, 5))
    sns.lineplot(data=trend, x=tcol, y="perc", hue="sintoma", marker="o", ax=ax)
    ax.set_title("Tendência Temporal de Sintomas (%)")
    ax.set_xlabel(tcol); ax.set_ylabel("Percentual (%)")
    savefig(fig, "tmp_tendencia_sintomas.png")

print(f"Gráficos salvos em: {OUT_DIR}")


<>:10: SyntaxWarning: invalid escape sequence '\G'
<>:10: SyntaxWarning: invalid escape sequence '\G'
C:\Users\dmaradini\AppData\Local\Temp\ipykernel_18620\4034816555.py:10: SyntaxWarning: invalid escape sequence '\G'
  OUT_DIR = "..\Graficos"


[WARN] https://techchallange-637423401077.s3.us-east-1.amazonaws.com/gold/eco_situacao_ocupacao.parquet -> HTTP 403
[INFO] Pulando eco_situacao_ocupacao (não encontrado em https://techchallange-637423401077.s3.us-east-1.amazonaws.com/gold/eco_situacao_ocupacao.parquet)
[WARN] https://techchallange-637423401077.s3.us-east-1.amazonaws.com/gold/eco_trabalhou_semana.parquet -> HTTP 403
[INFO] Pulando eco_trabalhou_semana (não encontrado em https://techchallange-637423401077.s3.us-east-1.amazonaws.com/gold/eco_trabalhou_semana.parquet)
[WARN] https://techchallange-637423401077.s3.us-east-1.amazonaws.com/gold/eco_teletrabalho.parquet -> HTTP 403
[INFO] Pulando eco_teletrabalho (não encontrado em https://techchallange-637423401077.s3.us-east-1.amazonaws.com/gold/eco_teletrabalho.parquet)
[WARN] https://techchallange-637423401077.s3.us-east-1.amazonaws.com/gold/eco_rendimento_base.parquet -> HTTP 403
[INFO] Pulando eco_rendimento_base (não encontrado em https://techchallange-637423401077.s3.us